In [16]:
# 패키지 설치
!pip install llama-index

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
# 환경 변수 준비
import os
os.environ["OPENAI_API_KEY"] = ""

In [19]:
# 로그 레벨 설정
# DEBUG로 설정 시 처리 내용이 상세하게 출력됨
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO, force=True)
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)

In [20]:
pip install llama-index-readers-database

In [21]:
pip install pymysql

In [22]:
# from llama_index.readers.database import DatabaseReader

# # DB 연결
# db = DatabaseReader(
#     scheme="mysql+pymysql",  # MySQL 연결
#     host="127.0.0.1",        # MySQL 서버 호스트
#     port=3306,             # MySQL 포트 번호
#     user="newuser",             # 사용자 이름
#     password="nurisung1023*",# 사용자 비밀번호
#     dbname="postcap",        # 데이터베이스 이름
# )

# # 정상 연결 테스트
# query = "SELECT * FROM contents"  # 실행할 SQL 쿼리
# texts = db.sql_database.run_sql(command=query)

# # 결과 출력
# print(type(texts))  # 결과 데이터 타입 확인
# print(texts)        # 쿼리 결과 출력

In [23]:
from llama_index.core import SimpleDirectoryReader

# 문서 로드
documents = SimpleDirectoryReader("/content/drive/MyDrive/postcap").load_data()

In [24]:
from llama_index.core import GPTVectorStoreIndex

# 인덱스 생성
index = GPTVectorStoreIndex.from_documents(documents)
# 인덱스 저장
index.storage_context.persist(persist_dir="./index")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [25]:
from llama_index.core import StorageContext, load_index_from_storage

# 재생성 storage context
storage_context = StorageContext.from_defaults(persist_dir="./index")

# index 가져오기
index = load_index_from_storage(storage_context)

INFO:llama_index.core.indices.loading:Loading all indices.


In [26]:
# Chat 엔진 생성
# friendly and colloquial language
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.llms.openai import OpenAI

memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
# llm = OpenAI(model="gpt-3.5-turbo")
llm = OpenAI(model="gpt-4-turbo")

chat_engine = index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    context_prompt=(
        "Recommend one or more companies.\n"
        "Do not mention prices unless I ask about them.\n"
        "Allow me to ask for detailed information.\n"
        "Respond by saying that information not in the data is confidential and to please contact the company for details.\n"

        "Every answer must be spoken in Korean.\n"
        "Here are the relevant documents for the context:\n"
        "{context_str}"
        "\nInstruction: Use the previous chat history, or the context above, to interact and help the user."
    ),
    verbose=False,
)

In [27]:
while True:
  user_input = input("Query: ")
  if user_input == "Q": break
  print(chat_engine.chat(user_input))

INFO:llama_index.core.chat_engine.condense_plus_context:Condensed question: 대학교와 협업을 진행한 회사 정보를 3개 정도 알려줘
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
1. 동국대학교 창업원과 협업한 Calmpictures
   - 프로젝트: 2021 DEW - Startup Campus Dongguk
   - 영상 링크: [동국대학교 창업원 영상](https://player.vimeo.com/video/878177318)
   - 협업 내용: 동국대학교 창업원의 2021년 연혁 및 성과를 공유하기 위한 인포그래픽 영상 제작

2. 중앙대학교 캠퍼스타운과 협업한 뭅스튜디오
   - 프로젝트: NET 포켓토닉
   - 영상 링크: [NET 포켓토닉 영상](https://player.vimeo.com/video/878177363)
   - 협업 내용: NET의 포켓토닉 홍보영상 제작, 제품의 다양한 활용성 강조

3. 중앙대학교 캠퍼스타운과 협업한 Calmpictures
   - 프로젝트: 메큐릭 닿기를 스프레이형 쇼츠
   - 영상 링크: [메큐릭 닿기를 스프레이형 쇼츠 영상](https://player.vimeo.com/video/878208992)
   - 협업 내용: 메큐릭의 유산균 Y존 케어 제품 닿기를 스프레이형 쇼츠 영상 제작

자세한 정보가 필요하시면 해당 회사에 직접 문의해 주시기 바랍니다.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.chat_engine.con